In [ ]:
import boto3
from datetime import datetime

# Print execution info
print(f"Execution started at: {datetime.now()}")

# Get current IAM role
sts = boto3.client('sts')
identity = sts.get_caller_identity()
print(f"Current Role ARN: {identity['Arn']}")
print(f"Account ID: {identity['Account']}")

In [ ]:
# Parameters cell - will be replaced by Papermill
agent_name = "calculator_agent"
agent_llm = "us.anthropic.claude-3-5-sonnet-20241022-v2:0"
force_recreate = True
kb_name = "mortgage-kb"

In [ ]:
# Cleanup from previous runs (ignore errors)
try:
    Agent.delete_by_name("mortgage_test_agent")
    kb_helper.delete_kb(kb_name, delete_s3_bucket=True, delete_iam_roles_and_policies=True)
    print("✅ Cleaned up previous resources")
except Exception as e:
    print(f"⚠️ Cleanup warning (may not exist): {e}")

In [ ]:
import os
print(f"IMAGE_VERSION: {os.environ.get('IMAGE_VERSION', 'Not set')}")
print(f"SAGEMAKER_INTERNAL_IMAGE_URI: {os.environ.get('SAGEMAKER_INTERNAL_IMAGE_URI', 'Not set')}")

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo
print(f"Execution Start Time (EST): {datetime.now(ZoneInfo('America/New_York')).strftime('%Y-%m-%d %H:%M:%S %Z')}")
print("version 1.0.0")

In [ ]:
# Package upgrade skipped - using environment default
print('Using default sagemaker_studio package from environment')

In [ ]:
import boto3
import time
from sagemaker_studio import Project

print("🚀 Starting Comprehensive Bedrock Agent Workflow")

# Get project connections dynamically (account-agnostic)
proj = Project()
region = boto3.Session().region_name
iam_conn = proj.connection('default.iam')
role = iam_conn.iam_role
s3_shared_conn = proj.connection('default.s3_shared')
bucket = s3_shared_conn.data.s3_uri.rstrip('/').split('/')[-2]

print(f"✅ Region: {region}")
print(f"✅ IAM Role: {role}")
print(f"✅ S3 Bucket: {bucket}")
print(f"✅ LLM: {agent_llm}")

In [ ]:
# Download utils module from S3
import subprocess
import sys
import os

print("\n📦 Downloading utils module from S3...")
s3_path = f's3://{bucket}/shared/genai/bundle/agent-code/utils/'
print(f"   S3 Path: {s3_path}")

result = subprocess.run([
    'aws', 's3', 'sync',
    s3_path,
    './utils/',
    '--region', region
], capture_output=True, text=True)

if result.returncode != 0:
    print(f"❌ S3 sync failed: {result.stderr}")
    raise Exception(f"Failed to download utils: {result.stderr}")

# Verify download
if os.path.exists('./utils'):
    files = os.listdir('./utils')
    print(f"✅ Utils module downloaded: {len(files)} files")
    for f in files:
        print(f"   - {f}")
else:
    print("❌ Utils directory not found!")
    raise Exception("Utils directory was not created")

# Add current directory to Python path
if '.' not in sys.path:
    sys.path.insert(0, '.')
    print("✅ Added current directory to sys.path")

In [ ]:
# Cleanup from previous runs
print("\n🧹 Cleaning up from previous runs...")

from utils.bedrock_agent import Agent
import boto3

# Delete agents from previous runs
agent_names = ["poet-agent", "calculator_agent", "test_agent", "test_user_data_agent", "mortgage_test_agent"]
for name in agent_names:
    try:
        if Agent.exists(name):
            Agent.delete_by_name(name)
            print(f"   Deleted agent: {name}")
    except Exception as e:
        print(f"   Could not delete {name}: {e}")

# Delete Lambda functions from previous runs
lambda_client = boto3.client('lambda', region_name=region)
lambda_names = ["mask_string", "mask_string_external", "transmorgifier", "get_user_data"]
for name in lambda_names:
    try:
        lambda_client.delete_function(FunctionName=name)
        print(f"   Deleted Lambda: {name}")
    except lambda_client.exceptions.ResourceNotFoundException:
        pass  # Function doesn't exist, that's fine
    except Exception as e:
        print(f"   Could not delete Lambda {name}: {e}")

print("✅ Cleanup complete")

## Scenario 1: Basic Agent Operations
Create, test existence, invoke, and delete a simple agent

In [ ]:
print("\n📝 Scenario 1: Basic Agent Operations")

Agent.set_force_recreate_default(force_recreate)

# Create simple poet agent
agent = Agent.create(
    "poet-agent",
    "You are a poet. You write short poems about the topics users provide.",
    llm=agent_llm
)

print("✅ Created poet-agent")

# Test existence
exists = Agent.exists("poet-agent")
print(f"✅ Agent exists: {exists}")

# Invoke agent
response = agent.invoke("Write a haiku about clouds")
print(f"✅ Agent response: {response}")

# Delete agent
Agent.delete_by_name("poet-agent")
print("✅ Deleted poet-agent")

## Scenario 3: Agent from YAML Template
Create an agent using a YAML configuration file

In [ ]:
print("\n📄 Scenario 3: Agent from YAML Template")

# Download YAML template from S3
import subprocess
yaml_path = './test_agent.yaml'
subprocess.run([
    'aws', 's3', 'cp',
    f's3://{bucket}/shared/genai/bundle/agent-code/test_agent.yaml',
    yaml_path,
    '--region', region
], check=True)

# Create agent from YAML (name, yaml_file)
test_agent = Agent.create_from_yaml('test_agent', yaml_path)
print("✅ Created test_agent from YAML template")

# Test agent
response = test_agent.invoke("Hello, how are you?")
print(f"✅ Agent response: {response[:100]}...")

## Scenario 4: Tools from Python Functions
Create and attach tools defined as Python functions

In [ ]:
print("\n🔧 Scenario 4: Tools from Python Functions")

# Define mask_string function
def mask_string(input_string: str) -> str:
    """Masks a string by replacing all characters with asterisks.
    
    Args:
        input_string: The string to mask
    
    Returns:
        A string of asterisks with the same length as input
    """
    return '*' * len(input_string)

# Attach tool to test_agent
test_agent.attach_tool_from_function(mask_string)
test_agent.prepare()

print("✅ Attached mask_string tool to test_agent")

# Test the tool
response = test_agent.invoke("Please mask the string 'secret123'")
print(f"✅ Tool response: {response}")

## Scenario 5: Tools via ParameterSchema
Create tools using ParameterSchema for external Lambda functions

In [ ]:
print("\n⚙️  Scenario 5: Tools via ParameterSchema")

from utils.bedrock_agent import Tool, ParameterSchema, ParamType

# Create parameter schema
schema = ParameterSchema.create_with_values(
    name="input_string",
    parameter_type=ParamType.STRING,
    description="The string to mask",
    required=True
)

# Create tool with schema and code file
mask_tool = Tool.create(
    "mask_string_external",
    schema=schema,
    code_file="lambda_mask_string.py",
    description="Masks a string by replacing characters with asterisks"
)

# Attach to agent
test_agent.attach_tool(mask_tool)
test_agent.prepare()

print("✅ Created and attached tool via ParameterSchema")

## Scenario 6: Tools Defined at Agent Creation
Define tools when creating the agent

In [ ]:
print("\n🛠️  Scenario 6: Tools Defined at Agent Creation")

# Define user data function
def get_user_data(user_id: str) -> dict:
    """Gets user data from a database.
    
    Args:
        user_id: The user ID to look up
    
    Returns:
        Dictionary with user information
    """
    return {"user_id": user_id, "name": "Test User", "email": "test@example.com"}

# Create agent first, then attach tool from function
test_user_data_agent = Agent.create(
    name="test_user_data_agent",
    role="You help users look up information",
    llm=agent_llm
)

# Attach tool from Python function
test_user_data_agent.attach_tool_from_function(get_user_data)
test_user_data_agent.prepare()

print("✅ Created agent and attached tool from function")

# Test the agent
response = test_user_data_agent.invoke("Get data for user ID 12345")
print(f"✅ Agent response: {response}")

# Cleanup
test_user_data_agent.delete()
print("✅ Deleted test_user_data_agent")

## Scenario 7: Knowledge Base Integration
Create and attach a knowledge base to an agent

In [ ]:
# Install opensearch-py for Knowledge Base scenario
%pip install -q opensearch-py